In [26]:
import numpy as np
import os
import pathlib
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import PIL

In [47]:
data_folder = "abc"
data_dir = pathlib.Path(f'{data_folder}/')                 
alphabats = list(data_dir.glob('*/*.jpg'))

batch_size = 64
img_height = 256
img_width = 256

In [48]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 6912 files belonging to 24 classes.
Using 4839 files for training.
Found 6912 files belonging to 24 classes.
Using 2073 files for validation.


In [49]:
class_names = train_ds.class_names
print(class_names)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


In [50]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(64, 256, 256, 3)
(64,)


In [51]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [52]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

0.18431373 0.92549026


In [53]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [54]:
num_classes = 24

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [55]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [56]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_7 (Rescaling)      (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 32, 32, 64)       

In [57]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
76/76 [==============================] - 323s 4s/step - loss: 2.8581 - accuracy: 0.1870 - val_loss: 2.2617 - val_accuracy: 0.3767
Epoch 2/10
76/76 [==============================] - 296s 4s/step - loss: 1.6427 - accuracy: 0.5753 - val_loss: 1.5208 - val_accuracy: 0.6228
Epoch 3/10
76/76 [==============================] - 275s 4s/step - loss: 0.9264 - accuracy: 0.7574 - val_loss: 1.3546 - val_accuracy: 0.6782
Epoch 4/10
76/76 [==============================] - 321s 4s/step - loss: 0.5750 - accuracy: 0.8533 - val_loss: 1.3166 - val_accuracy: 0.7096
Epoch 5/10
76/76 [==============================] - 284s 4s/step - loss: 0.3660 - accuracy: 0.9076 - val_loss: 1.2773 - val_accuracy: 0.7337
Epoch 6/10
76/76 [==============================] - 313s 4s/step - loss: 0.2423 - accuracy: 0.9401 - val_loss: 1.3731 - val_accuracy: 0.7352
Epoch 7/10
76/76 [==============================] - 294s 4s/step - loss: 0.1881 - accuracy: 0.9525 - val_loss: 1.6121 - val_accuracy: 0.7207
Epoch 8/10
76

In [58]:
model.evaluate(val_ds)

33/33 [==============================] - 26s 797ms/step - loss: 2.0067 - accuracy: 0.7361


[2.006711483001709, 0.7361311912536621]

In [59]:
model.save('modelweights.h5')

In [60]:
# model = tf.keras.models.load_model('modelweights.h5')

In [61]:
# ------------------------------ Use this code predict class -------------------------
import cv2
img_path = f"{data_folder}/A/3.jpg"
img_array = cv2.imread(img_path,1)
img_array = cv2.resize(img_array, (256,256))
img_array = tf.expand_dims(img_array, 0) # Create a batch
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to A with a 100.00 percent confidence.
